<a href="https://colab.research.google.com/github/manmal021/CPSMA3933_Labs_Manish/blob/main/lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation Assignment



In module 1, I proposed the problem of having 10 cases of water with 10 bottles in each case and 10 oz in each bottle.  One case is known to be filled with one ounce less than all the other bottles.  A former student proposed a random selection method for finding the case with the bottles that have less water.  Essentially you'll pull one bottle and test it.  If it is 9 ounces, you have found the case that is short.  Let's explore this random test.

1. Write a random function call that returns 10 ounces 9 out of 10 times and 9 ounces one out of ten.

2. What is the expected weight of a random pull?

3. Test your expected weight by running a simulation drawing at least 100 bottles and taking the average of the wieghts.  Does it agree with your theoretical result?

4. We are particuarly interested in how many pulls would be required to find the case that was short.  If you test one at a time, how many pulls do you expect before you find the case that is short?

5. Create a function that creates a sequence of pulling bottles and stops when you find the bottle with 9 ounces.  Test this function a bunch of times.  Do you notice anything odd when comparing it to your previous result?

6. Propose a modification to the coding you did in the first step that would improve the results.  Hint:  Consider a different data structure that would be more applicable to the real world case.

In [29]:


import random
import statistics
from collections import Counter

random.seed(1)


# 1) Random function: returns 10 oz 9/10 times, 9 oz 1/10 times

def pull_random_bottle():
    return 10 if random.random() < 0.9 else 9

# Quick smoke test
print("1) Examples of pulls:", [pull_random_bottle() for _ in range(10)])


# 2) Theoretical expected weight of a single random pull

p_10 = 0.9
p_9  = 0.1
E_theoretical = 10*p_10 + 9*p_9
print("\n2) Theoretical expected weight of a pull =", E_theoretical, "oz")

# 3) Simulation to test expected weight ( >= 100 pulls )

def simulate_avg_pull(n=1000):
    samples = [pull_random_bottle() for _ in range(n)]
    return sum(samples)/n

sim_avg_100  = simulate_avg_pull(100)
sim_avg_1000 = simulate_avg_pull(1000)
print("\n3) Simulation averages:")
print("   avg of 100 pulls  =", round(sim_avg_100, 4), "oz")
print("   avg of 1000 pulls =", round(sim_avg_1000, 4), "oz")
print("   (Theory = 9.9 oz) -> simulation agrees closely")

# 4)

p_short = 0.1
E_pulls_theory = 1 / p_short
print("\n4) Expected pulls (theoretical, geometric) =", E_pulls_theory)

# 5)
def pulls_until_short_singlepool():
    count = 0
    while True:
        count += 1
        if pull_random_bottle() == 9:
            return count

# Run many trials to estimate expected pulls and show distribution characteristics
trials = 20000
results = [pulls_until_short_singlepool() for _ in range(trials)]
mean_empirical = statistics.mean(results)
median_empirical = statistics.median(results)
p90 = sorted(results)[int(0.9*len(results))-1]
max_val = max(results)
counts = Counter(results)

print("\n5) Simulation of 'pull until 9-oz' (single-bottle model):")
print("   empirical mean over", trials, "trials =", round(mean_empirical,4))
print("   median =", median_empirical, "  90th percentile <=", p90, " max =", max_val)
print("   Note: empirical mean ≈ theoretical 10, but distribution has a long tail (sometimes large counts)")

# show probability mass for small n
print("   P(N=1..6) empirical:", {i: round(counts[i]/trials,4) for i in range(1,7)})


# 6)
E_cases_theory = (1 + 10) / 2

# Simulation of the case-by-case model:
def simulate_case_by_case(trials=20000):
    draws_needed = []
    for _ in range(trials):
        # create 10 cases; one random case is the short one
        cases = [10]*10   # we only need to mark which case is short
        short_index = random.randrange(10)
        # test cases in random order (simulate shuffle of cases)
        order = list(range(10))
        random.shuffle(order)
        # find position where short_index appears
        pos = order.index(short_index) + 1   # 1-based number of cases tested
        draws_needed.append(pos)
    return statistics.mean(draws_needed), draws_needed

mean_case_sim, draws_list = simulate_case_by_case(20000)
print("\n6) Improved model (10 cases, test cases one-by-one):")
print("   Theoretical expected number of case-tests =", E_cases_theory)
print("   Simulated mean (over 20000) =", round(mean_case_sim,4))
print("   Interpretation: if you test one bottle from a different case each trial,\n     you expect about 5.5 tests to find the short case (not 10).")

def simulate_random_case_pulls(trials=20000):
    # This sim samples a random case each pull (with replacement): success prob 1/10 each pull -> geometric mean=10
    results_rand = []
    for _ in range(trials):
        short_case = random.randrange(10)
        count = 0
        while True:
            count += 1
            # pick a random case and then pick a bottle from it (all bottles in that case are short if it's the short case)
            chosen_case = random.randrange(10)
            if chosen_case == short_case:
                results_rand.append(count)
                break
    return statistics.mean(results_rand)

def simulate_sequential_case_tests(trials=20000):
    # This sim shuffles cases and tests one bottle in each distinct case until short is found (without replacement)
    results_seq = []
    for _ in range(trials):
        short_case = random.randrange(10)
        order = list(range(10))
        random.shuffle(order)
        pos = order.index(short_case) + 1
        results_seq.append(pos)
    return statistics.mean(results_seq)

mean_rand_case = simulate_random_case_pulls(10000)
mean_seq_case = simulate_sequential_case_tests(10000)
print("\nExtra comparison:")
print("   If you pick a random case each pull (with replacement) -> empirical mean pulls ≈", round(mean_rand_case,4), "(≈10)")
print("   If you test cases sequentially without replacement -> empirical mean pulls ≈", round(mean_seq_case,4), "(≈5.5)")


print("\nFINAL SHORT ANSWERS:")
print("1) Random function provided: pull_random_bottle() (10 oz with prob .9, 9 oz with prob .1).")
print("2) Expected weight (theoretical) = 9.9 oz.")
print("3) Simulation (100 / 1000 pulls) gives averages near 9.9 oz; agrees with theory.")
print("4) If testing one-at-a-time with p=0.1 (geometric), expected pulls until 9-oz = 10.")
print("5) Simulation of pulling until 9-oz (single-bottle random model) -> empirical mean ≈ 10, but distribution has long tail (occasionally large counts).")
print("6) Improved real-world model: represent 10 cases and test one bottle per case (no replacement). Then expected case-tests = 5.5 (theoretical) — simulation confirms ≈5.5.")


1) Examples of pulls: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

2) Theoretical expected weight of a pull = 9.9 oz

3) Simulation averages:
   avg of 100 pulls  = 9.91 oz
   avg of 1000 pulls = 9.902 oz
   (Theory = 9.9 oz) -> simulation agrees closely

4) Expected pulls (theoretical, geometric) = 10.0

5) Simulation of 'pull until 9-oz' (single-bottle model):
   empirical mean over 20000 trials = 9.8987
   median = 7.0   90th percentile <= 22  max = 95
   Note: empirical mean ≈ theoretical 10, but distribution has a long tail (sometimes large counts)
   P(N=1..6) empirical: {1: 0.1017, 2: 0.089, 3: 0.0824, 4: 0.0727, 5: 0.0642, 6: 0.0579}

6) Improved model (10 cases, test cases one-by-one):
   Theoretical expected number of case-tests = 5.5
   Simulated mean (over 20000) = 5.4905
   Interpretation: if you test one bottle from a different case each trial,
     you expect about 5.5 tests to find the short case (not 10).

Extra comparison:
   If you pick a random case each pull (with rep

## Simulating Plates

In my household, there is a stack of plates that are differing in colors; pink, blue, black and green.  Of the 24 plates, 6 are each color.  My daughter will not eat dinner if it is not served on a pink plate.  

1.  Compute the probability that you draw a pink plate.
2. Compute the probability that of the four plates drawn for family dinner, atleast one of them is pink.
3. Create a randomization and test the previous two questions answers through randomization.  Discuss the results.
4. Let's make this a bit more complicated and indicative of how we live.  The plates are in a stack where the last one in, is the first one out.  Create a data structure that can account for this.
5. Also, we normally wash the plates when about 12 (sometimes more, sometimes less) collect in the dishwasher.  At that point, they are shuffled before returning to the stack in the cabinet.  Add this to your randomization and compute the probability of pulling exactly one and at least one pink plate when setting the table for four.
6. Since my daughter will not eat unless on the pink plate, most of the pink plates have migrated to the top of the stack.  This happened through a rejection of pull process.  In this, if we did not have at least one pink, more plates were removed from the stack until a pink was found.  Those rejected plates were then returned to the stack.  Create a function for doing this.
7. We have lived in the house for 1000 days.  Run your experiment on your stack always appeasing my daughter with at least one pink plate.  How often do you have all pink plates in the stack?
8. Explore what your final stack looks like by creating a usage count for each of the plates.  Have all the pink plates made it to the top and are some not in regular use?  

In [27]:
import random
from collections import Counter


# 1) Basic probabilities



p_single = 6/24


import math
def nCr(n,r): return math.comb(n,r)

p_at_least_one_analytical = 1 - nCr(18,4)/nCr(24,4)


# 2) Simulation for first part


def simulate_simple(trials=20000):
    at_least_one = 0
    exactly_one = 0
    deck = ["P"]*6 + ["B"]*6 + ["G"]*6 + ["K"]*6
    for _ in range(trials):
        sample = random.sample(deck,4)
        pinks = sample.count("P")
        if pinks >= 1: at_least_one += 1
        if pinks == 1: exactly_one += 1
    return at_least_one/trials, exactly_one/trials

sim_at_least_one, sim_exactly_one = simulate_simple()


# 3) Full household simulation


def simulate_house(days=1000, seed=1):
    random.seed(seed)

    # Create 24 unique plates (P/B/G/K)
    plates = []
    for color in ["P","B","G","K"]:
        for i in range(6):
            plates.append((color, f"{color}{i+1}"))

    # stack[-1] = top plate
    stack = plates.copy()
    random.shuffle(stack)

    dirty = []
    usage = Counter()
    top6_allpink = 0

    for _ in range(days):

        # Draw 4 plates (LIFO)
        served = []
        rejected = []

        for _ in range(4):
            served.append(stack.pop())

        # Ensure at least one pink - rejection rule
        while sum(1 for c,_ in served if c=="P") == 0:
            extra = stack.pop()
            rejected.append(extra)
            served.append(extra)
            if len(served) > 4:
                rejected.append(served.pop(0))

        # Put rejected back on top (LIFO)
        for plate in reversed(rejected):
            stack.append(plate)

        # Dirty plates and usage tracking
        for plate in served:
            dirty.append(plate)
            usage[plate[1]] += 1

        # Dishwasher: wash at 12 plates
        if len(dirty) >= 12:
            random.shuffle(dirty)
            stack.extend(dirty)
            dirty = []

        # Track top 6 all pink
        if len(stack) >= 6:
            top6 = stack[-6:]
            if all(c=="P" for c,_ in top6):
                top6_allpink += 1

    return {
        "top6_fraction": top6_allpink/days,
        "usage": usage,
        "final_top6": stack[-6:]
    }

results = simulate_house(1000, seed=3)




print("SHORT ANSWERS\n")

print("1) P(draw pink) =", round(p_single,4))
print("2) P(at least one pink in 4) =", round(p_at_least_one_analytical,4))

print("\nSimulation check (random sample of 4):")
print("   At least one pink =", round(sim_at_least_one,4))
print("   Exactly one pink =", round(sim_exactly_one,4))

print("\nAfter 1000 days with LIFO + rejection + dishwasher:")
print("   Fraction of days top-6 plates all pink =", round(results["top6_fraction"],4))

print("\nUsage counts (total uses per plate ID):")
for k,v in list(results["usage"].items())[:10]:
    print(k, v)
print("...(remaining plates skipped for shortness)")

print("\nFinal top-6 plates on the stack:")
print(results["final_top6"])


SHORT ANSWERS

1) P(draw pink) = 0.25
2) P(at least one pink in 4) = 0.712

Simulation check (random sample of 4):
   At least one pink = 0.7163
   Exactly one pink = 0.4674

After 1000 days with LIFO + rejection + dishwasher:
   Fraction of days top-6 plates all pink = 0.015

Usage counts (total uses per plate ID):
B2 26
K1 333
G6 421
P5 1
B6 333
G4 10
P3 625
P1 409
K5 5
K3 29
...(remaining plates skipped for shortness)

Final top-6 plates on the stack:
[('B', 'B6'), ('G', 'G6'), ('K', 'K1'), ('B', 'B3'), ('P', 'P1'), ('P', 'P4')]
